This code takes each IMERG rainfall file (coverage in a 1800 x 3600 x 1 3D array), converts to a MODIS system and fills a blank array with a given tiles data (H20V08 - 2400 x 2400 pixels)

In [1]:
import math
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import numpy.ma as ma
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.crs as crs
import cartopy.feature as cfeature
import h5py  
from astropy.time import Time
import os
import glob
from osgeo import gdal
import scipy.ndimage
import rasterio as rio
import re
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em
from pyhdf.SD import SD, SDC
import pprint
import matplotlib.colors as colors 
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import csv
import pprint
import matplotlib.colors as colors 
from mpl_toolkits.axes_grid1 import make_axes_locatable 

In [3]:
# Set input directory, and change working directory - plug in D:
inDir = "D:\\masters_data\\rainfall"   # I should change this so I can work from github?
os.chdir(inDir)  # Change to working directory
outDir = "D:\\masters_data\\rainfall\\H20_V08_arrays" # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [4]:
# select first MODIS file
IMERGFiles = glob.glob('*IMERG*')

In [5]:
# takes coordinate and returns corresponding index of IMEG list.
def latitude_index(x):
    index = round(((-89.95 - x)*-10), 1)
    index = int(index)
    return index

def longitude_index(x):
    index = round(((-179.95 - x)*-10), 1)
    index = int(index)
    return index

In [6]:
counter = 0
for i in IMERGFiles:
    IMERG = h5py.File(IMERGFiles[counter], 'r')
    precipitation = IMERG['/Grid/precipitation/'][:]
    precipitation = np.transpose(precipitation)
    latitude = IMERG['/Grid/lat'] [:]
    longitude = IMERG['/Grid/lon'][::]

    # create blank array with set value of -999
    rainfall_array = np.zeros((2400, 2400))
    rainfall_array[rainfall_array == 0] = -999

    # constants for conversion lat lon -> tile H,V and r,c
    R = 6371007.181	    # the radius of the idealizes sphere representing the Earth
    T = 1111950	        # the height and width of each MODIS tile in the projection plane
    Xmin = -20015109    # the western limit of the projection plane
    Ymax = 10007555	    # the northern limit of the projection plane
    w = T/2400.0	    # the actual size of a '500m' MODIS sinusoidal grid cell. (would be /1200 for a 1km grid)

    # select tile
    H = 20
    V = 8

    # this has calculated the lat and lon at the centre of the MODIS grid cells and stored them in a list
    lat = []
    lon = []

    for c in range(2400):
        for r in range(2400):
            X = (c + 0.5)*w + H*T + Xmin
            Y = Ymax - (r + 0.5)*w - V*T

            # Next compute the latitude φ and longitude λ at the center of the grid cell (in radians):
            PHI = Y / R
            LAMB = X / (R*math.cos(PHI))

             # convert to radians 
            X = PHI *(180/math.pi)
            Y = LAMB *(180/math.pi)

            lat.append(X)
            lon.append(Y)

    # Need coordinates of IMEG and MODIS to be stored in the same way (2dp - jumps in  0.05)
    lat_rounded = []
    lon_rounded = []

    for i in range(len(lat)):
        lat_rounded.append(round(round(lat[i]/0.05)*0.05, 2))
    for i in range(len(lon)):
        lon_rounded.append(round(round(lon[i]/0.05)*0.05, 2))

    # creates list of IMEG precipitation values based on MODIS coordinates
    precip_values = []
    for i in range(2400*2400):
        lat_number = lat_rounded[i]
        lon_number = lon_rounded[i]

        array_lat = latitude_index(lat_number)
        array_lon = longitude_index(lon_number)
        
        precip = precipitation[array_lat][array_lon][0]
        precip_values.append(precip *24*365.25) # plots it in mm/year

    precip_array = np.array(precip_values)
    precip_array = precip_array.reshape(2400,2400)

    File = IMERGFiles[counter]
    FileName = File.split(".")[4][0:4] + "_" + File.split(".")[4][4:6] + "_" + File.split(".")[4][6:8] + "_rainfall"

    np.savetxt(("D:\\masters_data\\output\\csv\\Rainfall\\H20_V08_arrays\\" + FileName +".csv"), precip_array, delimiter=",")
    counter +=1